In [37]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pytube import YouTube
from pymongo import MongoClient
from tqdm import tqdm
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import urllib.request
import datetime
import copy

In [38]:
# MongoDB 클라이언트 및 컬렉션 설정
client = MongoClient(host="1.234.51.110", port=38019, username='clawling', password='goodtime**95')
db = client['YouTube_crawling']
collection = db['OneAsia_YouTube_crawling']
data_list = list(collection.find({}, {"_id": 1, "URL": 1}))

In [40]:
data_list1 = copy.deepcopy(data_list)

In [41]:
driver = webdriver.Chrome()

In [ ]:
# 좋아요 개수를 변환 (ex) 1.3천 → 1300, 1.3만 → 13000 형식으로 변환)
def convert_like_count(like_count_text):
    if '천' in like_count_text:
        count = int(float(like_count_text.replace('천', '')) * 1000)
    elif '만' in like_count_text:
        count = int(float(like_count_text.replace('만', '')) * 10000)
    else:
        count = int(like_count_text)
    return count

In [42]:
def process_link(url):
    try:
        driver.get(url)

        time.sleep(3)

        # 설명 요소 찾기
        more_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "expand")))
        more_button.click()
        time.sleep(2)

        description_element = driver.find_element(By.XPATH, '//div[@id="description-inner"]//yt-attributed-string[@class="style-scope ytd-text-inline-expander"]')
        description_text = description_element.text if description_element else ""

        print("Description Text:", description_text)

        # 맨 아래까지 스크롤
        SCROLL_PAUSE_TIME = 1.5
        last_height = driver.execute_script("return document.documentElement.scrollHeight")

        while True:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.documentElement.scrollHeight;")
            if new_height == last_height:
                break
            last_height = new_height
        
        time.sleep(3)

        # 좋아요 요소 찾기 
        try:
            like_button = driver.find_element(By.CSS_SELECTOR, '.yt-spec-button-shape-next--icon-leading[aria-label^="나 외에 사용자"]')
            like_count_text_element = like_button.find_element(By.CSS_SELECTOR, '.yt-spec-button-shape-next__button-text-content')
            like_count_text = like_count_text_element.text
        except Exception as e:
            like_count_text = ""
            print("좋아요 수 텍스트를 가져오는 중 오류 발생:", e)

        # 숫자형으로 변환
        if like_count_text:
            like_count = convert_like_count(like_count_text)
        # 좋아요 개수 없을 경우 0을 할당
        else:
            like_count = 0

        print("Like Count:", like_count)

        # Mongodb에 적재
        collection.update_one({"URL": url}, {"$set": {
            "설명": description_text,
            "좋아요": like_count,
        }})

    except Exception as e:
        print("URL 처리 중 오류 발생:", e)

In [43]:
for data in tqdm(data_list1, desc="Processing Links", unit="link"):
    url = data['URL']
    process_link(url)

client.close()
driver.quit()

Processing Links:   0%|          | 0/444 [00:00<?, ?link/s]

Description Text: #Kstyle #TREASURE #트레저

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝

21日午後、釜山（プサン）アジアド主競技場で「釜山ワンアジアフェスティバル 2023（BUSAN ONE ASIA FESTIVAL 2023）」のK-POPコンサートが開催✨

TREASUREの「JIKJIN」のステージをお届け！

・フォト記事はコチラ
⇒https://news.kstyle.com/article.ksn?a...

・TREASURE 関連記事はコチラ
⇒https://news.kstyle.com/topicNews.ksn...

★Kstyleで最新ニュースをチェック！
⇒https://www.kstyle.com/

★Kstyle公式SNS
Twitter：
  / kstyle_news  
Instagram：
  / kstylenews  
★KstyleのLINE公式アカウント
【友だち追加はコチラ】⇒ https://line.me/R/ti/p/@oa-kstyle?fro...

＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝


Processing Links:   0%|          | 1/444 [00:12<1:34:47, 12.84s/link]

Like Count: 177
Description Text: 「2017 釜山ワンアジアフェスティバル 開幕記念公演」
韓流を代表するスターだけが立てる舞台。K-POPを率いてきた最高のアーティスト達が集結！

【放送日】 12月23日（土）19：00～
出演：ナムグン・ミン、キム・ソヒョン、Brave Brother、Apink、ASTRO、BAP、GOT7、SF9、NU'EST W、MOMOLAND、BLACKPINK、iKON、GFRIEND、Wanna One、SECHSKIES、NCT127ほか

番組サイト→https://knda.tv/datv/program/dt001283/


Processing Links:   0%|          | 2/444 [00:23<1:25:47, 11.65s/link]

...

Like Count: 4700
Description Text: Lying is technically not a virtue, but when you put NCT 127 at a round table and make them trick each other, it makes for great entertainment 🤣

Afterwards, you can get your “Fact Check” on Spotify’s K-Pop ON! Playlist! 

Listen to K-Pop ON! Playlist, only on Spotify 🎧
👉👉 https://spotify.link/K-PopON 👈👈

Watch K-Pop ON! Video Podcast, only on Spotify 🖥️
👉👉 https://spotify.link/kpoponpodcast 👈👈

#NCT127 #NCT127_FactCheck #NCT127_FactCheck_불가사의


Processing Links: 100%|█████████▉| 443/444 [1:44:40<00:22, 22.82s/link]

Like Count: 33000
Description Text: よければぜひチャンネル登録よろしくお願いいたします！！
Thanks for watching! Please subscribe to the channel if you like!

🌍ワールドワイドに顎を外します🌏
          
毎週 火・木 ・土20:00に動画投稿‼️
ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー
アポバラの待合室(サブチャンネル)
▶
   / @apovara_no_machiaishitu  
公式tapnow
▶https://tapno.onelink.me/pJnR/3j8t43qi

公式Twitter
▶
  / apollonvariety  
公式Instagram
▶
  / apollonvariety_official  
公式グッズ
▶https://apovara.official.ec
ーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーーー

【メンバー】

💖ユート(YUTO)ーーーーーーー
●Twitter
▸
  / yutopiapollon  
●Instagram
▸
  / yuto_yt_1106  
💚ジョリ(Jori)ーーーーーーーー
●Twitter
▸
  / jori_apovara  
●Instagram
▸
  / jori_mm  
💜カンチー(Kanchi)ーーーーー
●Twitter
▸
  / apovara_kanchii  
●Instagram
▸
  / apovara_kanchii  
❤️トイ(ToY)ーーーーーーーーー
●Twitter
▸
  / apovara_toy  
●Instagram
▸
  / toypoo_apovara  
#NCT127 #엔시티127 #FactCheck #불가사의 #不可思議
#NCT127_FactCheck
#NCT127_FactCheck_불가사의
#FactCheck_불가사의_不可思議

(編集：トイ)


Processing Links: 100%|██████████| 444/444 [1:44:52<00:00, 14.17s/link]

Like Count: 413
